In [198]:
import pandas
import os
from os import listdir
from os.path import isfile, join
import re

In [199]:
targetDir = '/Users/Enya/Desktop/content-analysis-2018/data/new_york_times'

In [221]:
def getText(x):
    reResults = re.findall(r'(DOCUMENTS)(.*?)((Copyright)\s+(\d{4}))', x.replace('\n', ' '), flags = re.M)
    return[t[1] for t in reResults]

def getDate(x) :
    reResults = re.findall(r'(LOAD-DATE)(.*?)(LANGUAGE)', x.replace('\n', ' '))
    return[t[1] for t in reResults]

In [222]:
nytText = []
nytDate = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path, encoding='utf-8') as f:
        workingText = f.read()
        workingText = str(workingText)
        goodText = getText(workingText)
        date = getDate(workingText)
        nytText += goodText
        nytDate += date

nytDF = pandas.DataFrame({'date' : nytDate, 'article' : nytText})

In [223]:
print(nytDF)

                                                article                   date
0                                    The New York Ti...    : January 6, 2014  
1                                       The New York...   : January 10, 2014  
2                                       The New York...   : January 15, 2014  
3                                       The New York...   : January 17, 2014  
4                                       The New York...   : January 18, 2014  
5                                       The New York...   : January 22, 2014  
6                                       The New York...   : January 20, 2014  
7                                       The New York...   : January 20, 2014  
8                                    The New York Ti...   : January 21, 2014  
9                                    The New York Ti...   : January 22, 2014  
10                                      The New York...   : January 24, 2014  
11                                   The New York Ti

In [224]:
def cleanerUpper(x):
    return re.sub(r'\s+NEW YORK TIMES\s+', '', x)

# strip out NYT title lower case
def cleanerLower(x):
    return re.sub(r'\s+The New York Times\s+', '', x)

#strip out useless text
def cleanerText(x):
    return re.sub(r'(January|February|March|April|May|June|July|August|September|October|November|December)(.*?)(LENGTH:)', '', x)

# strip out copyright and year
def cleanerCopy(x):
    return re.sub(r'(Copyright)\s+(\d{4})', '', x)

# simply the date
def cleanerYear(x):
    return re.sub(r'(\D+)\s(\d{1,2})\W', '', x)

# behaves as advertised - destroys first comma found
def destroyComma(x):
    return re.sub(r',', '', x)

# strip word count
def cleanWordCount(x):
    return re.sub(r'(\d+).(words)', '', x)

# death to datelines
# not that clean. What does 25835 look like before this function?
def cleanDateline(x):
    return re.sub(r'(DATELINE:)\s(\w+)(,\s(\w+).\s(\d+))?', '', x)

In [225]:
nytDF['date'] = nytDF['date'].apply(cleanerUpper)
nytDF['date'] = nytDF['date'].apply(cleanerLower)
#nytDF['date'] = nytDF['date'].apply(cleanerYear)
nytDF['date'] = nytDF['date'].apply(destroyComma)

#clean articles
nytDF['article'] = nytDF['article'].apply(cleanerUpper)
nytDF['article'] = nytDF['article'].apply(cleanerLower)
nytDF['article'] = nytDF['article'].apply(cleanerText)
nytDF['article'] = nytDF['article'].apply(cleanerCopy)
nytDF['article'] = nytDF['article'].apply(cleanWordCount)
nytDF['article'] = nytDF['article'].apply(cleanDateline)

In [226]:
print(nytDF)

                                                article                  date
0     Blogs                                      (Bi...    : January 6 2014  
1         Gabriel Vaughn, the commando turned human ...   : January 10 2014  
2         WASHINGTON -- The National Security Agency...   : January 15 2014  
3         TOKYO -- An arms race between the two most...   : January 17 2014  
4         Following is the text of President Obama's...   : January 18 2014  
5         WASHINGTON -- Google, which briefly consid...   : January 22 2014  
6         If the Air Force cheating scandal disclose...   : January 20 2014  
7         WASHINGTON -- The heads of the House and S...   : January 20 2014  
8     Blogs                                (You're t...   : January 21 2014  
9     Blogs                                      (Bi...   : January 22 2014  
10        WASHINGTON -- A three-year study by the Pe...   : January 24 2014  
11    Blogs                                    (Deal...   : Janu

In [233]:
nytDF.to_csv('nytFull', index=False, header=True)